# Set up

In [ ]:
import io
import pprint
import requests
import json
from keycloak import KeycloakAdmin
from keycloak import KeycloakOpenID

APP_BASE_URL='http://localhost:8080'
APP_ADMIN_USERNAME='admin@everest.engineering'
APP_ADMIN_PASSWORD='ac0n3x72'

KEYCLOAK_BASE_URL='http://localhost:8180'
KEYCLOAK_ADMIN_USERNAME='admin@everest.engineering'
KEYCLOAK_ADMIN_PASSWORD='ac0n3x72'

# This needs to be generated in Keycloak for the 'default-client' in the 'default' realm. See the top level README.md
OID_CLIENT_SECRET='3b589b78-3280-43d4-ab32-b7e37b40781a'

pp = pprint.PrettyPrinter()

oid_client = KeycloakOpenID(
    server_url=f"{KEYCLOAK_BASE_URL}/auth/",
    realm_name="default",
    client_id="default-client",
    client_secret_key=OID_CLIENT_SECRET,
    verify=False)

## Application admin login

In [ ]:
admin_tokens = oid_client.token(APP_ADMIN_USERNAME, APP_ADMIN_PASSWORD)
pp.pprint(admin_tokens)

In [ ]:
admin_userinfo = oid_client.userinfo(admin_tokens['access_token'])
pp.pprint(admin_userinfo)

# System info

## OID configuration

In [ ]:
pp.pprint(oid_client.well_know())

## Application version

In [ ]:
print(requests.get(f'{APP_BASE_URL}/api/version').text)

## Health status

In [ ]:
public_health = requests.get(f'{APP_BASE_URL}/actuator/health')
pp.pprint(public_health.json())

In [ ]:
private_health = requests.get(
    f'{APP_BASE_URL}/actuator/health', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(private_health.json())

## Available system metrics

In [ ]:
metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(metrics.json()['names'])

## HTTP server request metrics

In [ ]:
request_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics/http.server.requests', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(request_metrics.json())

## Prometheus metrics

In [ ]:
prometheus_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/prometheus', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(prometheus_metrics.text)

## Replay status

In [ ]:
replay_status = requests.get(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(replay_status.json())

## Trigger replay

In [ ]:
response = requests.post(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    },
    json={}
)
# You are expecting a 204 NO-CONTENT response here
pp.pprint(response.status_code)

# Self registration scenario

## Simulated user self registration for Bob

In [ ]:
keycloak_admin = KeycloakAdmin(
    server_url=f"{KEYCLOAK_BASE_URL}/auth/",
    username=KEYCLOAK_ADMIN_USERNAME,
    password=KEYCLOAK_ADMIN_PASSWORD,
    realm_name='default',
    user_realm_name='master',
    verify=False)

In [ ]:
bob_user_id = keycloak_admin.create_user(
    {
        "email": "bob@example.com",
        "username": "bob@example.com",
        "enabled": True,
        "firstName": "Bob",
        "lastName": "Example",
        "credentials": [{"value": "password-here", "type": "password"}]
    }
)         
pp.pprint(bob_user_id)

## Bob's user and organisation does not exist until first use

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/users',
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(response.json())

In [ ]:
response = requests.get(f'{APP_BASE_URL}/admin/organizations',
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(response.json())

## Bob logs in

In [ ]:
bob_tokens = oid_client.token('bob@example.com', 'password-here')
pp.pprint(bob_tokens)

## Bob retrieves profile details

In [ ]:
# After this call you can go back up and check - the user and their default organisation should now be available
response = requests.get(f'{APP_BASE_URL}/api/user',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
).json()
pp.pprint(response)
bob_org_id = response['organizationId']

## Bob uploads profile photo

In [ ]:
with open('profile.jpg', 'rb') as image:
    response = requests.post(f'{APP_BASE_URL}/api/user/profile-photo',
        headers=
        {
            "Authorization": f"Bearer {bob_tokens['access_token']}"
        },
        files= 
        {
            'file': image
        }
    )
    print(response.status_code)

## Bob downloads profile photo

In [ ]:
from IPython.display import Image
response = requests.get(f'{APP_BASE_URL}/api/user/profile-photo/thumbnail?width=127&height=128',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
Image(data=response.content)

## Admin deletes user and discards encryption key

In [ ]:
# After this call you can go back up and check that replays still work and that 
# Bob's information no longer appears anywhere in the API or event log
response = requests.post(f'{APP_BASE_URL}/api/users/{bob_user_id}/forget',
    headers=
    {
        'Authorization': f"Bearer {admin_tokens['access_token']}"
    },
    json={
      'requestReason': 'GDPR request received'
    }                  
)
print(response)